# Tarea 1 Aprendizaje Automático UCV 02-2025
#### Jhonatan Homsany C.I. 30.182.893
#####Regresión logística.

# *Cáncer de mama.*

# Carga de datos

In [21]:
!pip install ucimlrepo
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_breast_cancer
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets
print(breast_cancer_wisconsin_diagnostic.metadata)
print(breast_cancer_wisconsin_diagnostic.variables)

{'uci_id': 17, 'name': 'Breast Cancer Wisconsin (Diagnostic)', 'repository_url': 'https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic', 'data_url': 'https://archive.ics.uci.edu/static/public/17/data.csv', 'abstract': 'Diagnostic Wisconsin Breast Cancer Database.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 569, 'num_features': 30, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Diagnosis'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1993, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5DW2B', 'creators': ['William Wolberg', 'Olvi Mangasarian', 'Nick Street', 'W. Street'], 'intro_paper': {'ID': 230, 'type': 'NATIVE', 'title': 'Nuclear feature extraction for breast tumor diagnosis', 'authors': 'W. Street, W. Wolberg, O. Mangasarian', 'venue': 'Electronic imaging', 'year': 1993, 'journal': None, 'DOI': '1

#Modelo

In [22]:
def funcion_logistica(x, L, k, x0):
    return 1/(1.0 + np.exp(-1.0*k*(x-x0)))

In [23]:
le = LabelEncoder()
y_numeric = le.fit_transform(y.values.ravel())

scaler_final = StandardScaler()
X_all_scaled = scaler_final.fit_transform(X)

model = LogisticRegression(max_iter=1000, random_state=42).fit(X_all_scaled, y_numeric)

k = model.coef_[0]
beta_0 = model.intercept_[0]
print("B_0 =", beta_0)
print("K=", k)

B_0 = -0.22088610964220162
K= [ 0.37545731  0.38203064  0.36097105  0.43946905  0.16737642 -0.56070701
  0.85529546  0.96284928 -0.07631246 -0.3284581   1.28926139 -0.26511648
  0.6718954   0.99886093  0.27950865 -0.74407728 -0.1012066   0.32314492
 -0.29504386 -0.68174616  1.02658552  1.32055634  0.82039667  0.99467101
  0.66571519 -0.05114225  0.88022314  0.92517516  0.88871131  0.48699036]


# Prueba del modelo

In [24]:
data = load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)

print(f"Precisión media (CV): {cv_scores.mean():.4f}")

model.fit(X_train_scaled, y_train)
print(f"Calidad del modelo en test: {model.score(X_test_scaled, y_test):.4f}")

Precisión media (CV): 0.9748
Calidad del modelo en test: 0.9825


# Técnicas de selección de muestras representativas.

Para la selección de $k$ mujeres en un universo $N$ que nos otorgue una muestra representativa podemos utilizar la estrategia de la muestra estratificada propuesta por Neyman J. ($1934$).

La muestra estratificada consiste en dividir a una población en $m$ grupos a partir de una característica específica y luego tomar $k/m$ elementos aleatorios de cada grupo. La característica a elegir es un factor muy dependiente del problema bajo estudio. En esta ocasión, la característica a elegir podría ser una que genere la mayor cantidad de grupos posibles (así separamos a más personas con características diferentes) y que más influya en la decisión de si el cáncer que se padece es benigno o maligno.

# Estandarización de las variables.

En problemas como el que se nos presenta, ocurre que la diferencia entre los valores es considerable pero esto no necesariamente implica que una variable es más importante que otra. Por esta razón, la estandarización de variables es una necesidad para tener resultados consistentes en nuestro modelo. El proceso de estandarización consiste en reducir los valores a una misma escala que evite que el modelo adquiera un sesgo hacia variables con valores más altos por considerarlos significativos.

#Interpretación de los coeficientes.

In [25]:
coef_df = pd.DataFrame({
    'Variable': data.feature_names,
    'Coeficiente': model.coef_[0]
})

coef_df['Odds_Ratio'] = np.exp(coef_df['Coeficiente'])
coef_df['Abs_Coef'] = np.abs(coef_df['Coeficiente'])
top_5 = coef_df.sort_values(by='Abs_Coef', ascending=False).head(5)
print("Análisis de los 5 coeficientes más importantes y sus Odds Ratios:")
print(top_5[['Variable', 'Coeficiente', 'Odds_Ratio']])

Análisis de los 5 coeficientes más importantes y sus Odds Ratios:
               Variable  Coeficiente  Odds_Ratio
21        worst texture    -1.291134    0.274959
10         radius error    -1.244185    0.288176
28       worst symmetry    -1.202497    0.300443
7   mean concave points    -1.103646    0.331660
26      worst concavity    -0.974874    0.377240


En términos médicos, un coeficiente positivo indica que un coeficiente aporta a que el diagnóstico de la patología bajo estudio en un paciente sea positivo. En este caso, un coeficiente positivo indicaría que la persona padece de un tumor maligno mientras que un coeficiente negativo indicaría que el tumor es benigno.

Los odds representan la probabilidad de que un evento ocurra frente a la posibilidad de su complemento. A diferencia del modelo de Laplace, el odd ratio no evalúa los casos favorables sobre los casos totales sino que calcula la proporción de los casos favorables sobre los casos no favorables. Matemáticamente:

$$\dfrac{|E|}{|E^c|}$$

Por otro lado, los odds ratios (OR) se calculan como $e^\beta$ y representan cuánto cambia la proporción en función del aumento unitario de una variable. En este escenario, si $OR > 1$, el aumento de la variable aumenta la posibilidad de un tumor maligno. Si $OR = 1$, la variable no influye en la predicción y un $OR < 1$ indica que el aumento de la variable aumenta la posibilidad de que el tumor sea benigno.

Ahora, evaluando los cinco coeficientes más informativos que obtuvimos con el algoritmo anterior, tenemos que:

* Worst texture $(OR ≈ 0.275)$: Por cada unidad que aumenta esta variable, la probabilidad de que el tumor sea maligno se reduce en aproximadamente un $72.5\%~(1 - 0.275)$. Es el predictor más fuerte de tu lista.

* Radius error $(OR ≈ 0.288)$: Un incremento unitario en el error del radio reduce las posibilidades de malignidad en un $71.2\%$.

* Worst symmetry $(OR ≈ 0.300)$: La probabilidad de que el tumor sea maligno es solo el $30\%$ de lo que era antes por cada unidad de aumento en la simetría "peor".

* Mean concave points $(OR ≈ 0.332)$: El riesgo de malignidad cae un $66.8\%$ ante un aumento unitario en el promedio de puntos cóncavos.

* Worst concavity $(OR ≈ 0.377)$: Es el quinto factor más influyente; un aumento reduce el riesgo de cáncer maligno en un $62.3\%$.

# Desbalance de clases.

El desbalance de clases en este problema provocaría que nuestro modelo se ajuste demasiado a las clases de las que tenemos mayor información. Esto provocaría que el modelo tenga una tendencia a clasificar nuestros vectores como vectores de la clase más presente en el conjunto de entrenamiento. Una estrategia que puede ser empleada para mitigar el problema es la propuesta por Lin Et Al (2017) llamada "pérdida focal", donde establece un factor de modulación dicional que disminuye el efecto que tienen las clases bien representadas en el modelo para mejorar el rendimiento. Además, tenemos la técnica del resampleo, dónde se submuestrea a la clase bien representada y se sobremuestrea a la clase menos representada para así provocar el balance y mejorar el rendimiento del modelo.

# Umbral de clasificación.

Estamos ante un problema de clasificación binaria, ya que el modelo calcula la probabilidad de que un vector de características $X$ pertenezca a una de dos categorías del conjunto $C = \{benigno, maligno\}$. Por lo tanto, el umbral de decisión se define en el intervalo $[0, 1]$, donde valores cercanos a $1$ indican una alta probabilidad de malignidad según la función logística.

En el contexto médico, existe una asimetría en el costo del error. Un falso negativo es crítico, pues implica informar a una paciente que está sana cuando en realidad padece una enfermedad potencialmente mortal, retrasando su tratamiento. Por el contrario, un falso positivo implica que una persona sana sea sometida a exámenes adicionales y biopsias; aunque esto conlleva un costo económico y estrés emocional, no pone en riesgo la vida de la paciente.

Por este motivo, se recomienda establecer un umbral de clasificación conservador (por debajo de 0.5). Con esto, el modelo arrojará más resultados de tumores malignos de los que existen realmente en el dataset, maximizando la sensibilidad del sistema para asegurar que ningún caso real pase desapercibido.